In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/previous-applications-merged/previous_merged.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/previous-applications-imputed/previous_merged_imputed.csv
/kaggle/input/previous-loans-short/previous_loans_short.csv
/kaggle/input/bureau-kaggle/bureau_kaggle.csv
/kaggle/input/previous-kaggle/previous_loans_int.csv


In [13]:
df_application = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
df_bureau_balance = pd.read_csv("/kaggle/input/bureau-kaggle/bureau_kaggle.csv")
df_previous = pd.read_csv("/kaggle/input/previous-kaggle/previous_loans_int.csv")

y = df_application[['TARGET', 'SK_ID_CURR']]
y=y.astype(int)
df_application=df_application.drop('TARGET',axis=1)


y.shape
df_application.shape



(307511, 121)

In [14]:
def force_optimize_all_to_int(df):
    for col in df.columns:
        col_data = df[col]

        # Skip non-numeric columns
        if not pd.api.types.is_numeric_dtype(col_data):
            continue

        # If float → round and convert to int64 first
        if pd.api.types.is_float_dtype(col_data):
            col_data = col_data.round().astype('int64')

        # Handle integer downcasting
        col_min = col_data.min()
        col_max = col_data.max()

        if col_min >= -128 and col_max <= 127:
            df[col] = col_data.astype('int8')
        elif col_min >= -32_768 and col_max <= 32_767:
            df[col] = col_data.astype('int16')
        elif col_min >= -2_147_483_648 and col_max <= 2_147_483_647:
            df[col] = col_data.astype('int32')
        else:
            df[col] = col_data.astype('int64')  # leave as is if very large

    return df


In [15]:
print("Before:")
print(df_previous.memory_usage(deep=True).sum() / 1024**2, "MB")

df_previous = force_optimize_all_to_int(df_previous)

print("After:")
print(df_previous.memory_usage(deep=True).sum() / 1024**2, "MB")


Before:
2194.897533416748 MB
After:
402.97967052459717 MB


In [16]:
categorical_columns = df_application.select_dtypes(exclude = 'number').columns
df_categorical = df_application[categorical_columns].copy()
categorical_values = df_categorical.nunique()


In [17]:
label = []
hot = []

for i in range(0,len(categorical_values)):
    if categorical_values.iloc[i]>2:
        hot.append(categorical_values.index[i])
    else:
        label.append(categorical_values.index[i])

In [18]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

hot_encoder = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)
encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

hot_encoded_features = pd.DataFrame(hot_encoder.fit_transform(df_categorical[hot]),columns=hot_encoder.get_feature_names_out(hot))
ordinal_encoded_features = pd.DataFrame(encoder.fit_transform(df_categorical[label]),columns=label)

df_categorical = pd.concat([hot_encoded_features,ordinal_encoded_features],axis=1)
df_categorical

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,...,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
307507,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
307508,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
307509,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
numerical_columns = df_application.select_dtypes(include='number').columns
df_numerical = df_application[numerical_columns].copy()

df = pd.concat([df_numerical,df_categorical],axis=1)
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# impute numerical missing values by first finding out the columns where more than 40% of rows are empty

# Step 1 : Drop useless features

empty_columns=[]
"""
for column in df.columns:
    if df[column].isnull().sum()/307511>0.4:
        empty_columns.append(column)

df= df.drop(columns=empty_columns,axis=1)

print(empty_columns)
print(len(empty_columns))
"""

'\nfor column in df.columns:\n    if df[column].isnull().sum()/307511>0.4:\n        empty_columns.append(column)\n\ndf= df.drop(columns=empty_columns,axis=1)\n\nprint(empty_columns)\nprint(len(empty_columns))\n'

In [21]:
df.isnull().sum()

SK_ID_CURR                     0
CNT_CHILDREN                   0
AMT_INCOME_TOTAL               0
AMT_CREDIT                     0
AMT_ANNUITY                   12
                           ...  
WALLSMATERIAL_MODE_nan         0
NAME_CONTRACT_TYPE             0
FLAG_OWN_CAR                   0
FLAG_OWN_REALTY                0
EMERGENCYSTATE_MODE       145755
Length: 246, dtype: int64

In [22]:
# Step 2 : Impute remaining features
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=0)
df = pd.DataFrame(imp_mean.fit_transform(df),columns = df.columns)
df.shape

(307511, 246)

In [23]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = df[[ 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
poly_transformer = PolynomialFeatures(degree = 3)
poly_transformer.fit(poly_features)
poly_features = poly_transformer.transform(poly_features)
poly_transformer.get_feature_names_out(input_features = ['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names_out([ 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))



In [24]:
#df=df.drop(columns=['EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'],axis=1)
poly_features.shape

(307511, 20)

In [25]:
poly_features

,1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,EXT_SOURCE_2^2,EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_2 DAYS_BIRTH,EXT_SOURCE_3^2,EXT_SOURCE_3 DAYS_BIRTH,DAYS_BIRTH^2,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
0,1.0,0.262949,0.139376,-9461.0,0.069142,0.036649,-2487.756636,0.019426,-1318.634256,89510521.0,0.018181,0.009637,-654.152107,0.005108,-346.733022,2.353667e+07,0.002707,-183.785678,1.247560e+07,-8.468590e+11
1,1.0,0.622246,0.000000,-16765.0,0.387190,0.000000,-10431.950422,0.000000,-0.000000,281065225.0,0.240927,0.000000,-6491.237078,0.000000,-0.000000,1.748916e+08,0.000000,-0.000000,0.000000e+00,-4.712058e+12
2,1.0,0.555912,0.729567,-19046.0,0.309038,0.405575,-10587.901540,0.532268,-13895.327191,362750116.0,0.171798,0.225464,-5885.942404,0.295894,-7724.580288,2.016572e+08,0.388325,-10137.567875,2.646504e+08,-6.908939e+12
3,1.0,0.650442,0.000000,-19005.0,0.423074,0.000000,-12361.644326,0.000000,-0.000000,361190025.0,0.275185,0.000000,-8040.528832,0.000000,-0.000000,2.349331e+08,0.000000,-0.000000,0.000000e+00,-6.864416e+12
4,1.0,0.322738,0.000000,-19932.0,0.104160,0.000000,-6432.819536,0.000000,-0.000000,397284624.0,0.033616,0.000000,-2076.117157,0.000000,-0.000000,1.282190e+08,0.000000,-0.000000,0.000000e+00,-7.918677e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,1.0,0.681632,0.000000,-9327.0,0.464623,0.000000,-6357.585111,0.000000,-0.000000,86992929.0,0.316702,0.000000,-4333.535804,0.000000,-0.000000,5.929720e+07,0.000000,-0.000000,0.000000e+00,-8.113830e+11
307507,1.0,0.115992,0.000000,-20775.0,0.013454,0.000000,-2409.735313,0.000000,-0.000000,431600625.0,0.001561,0.000000,-279.510194,0.000000,-0.000000,5.006225e+07,0.000000,-0.000000,0.000000e+00,-8.966503e+12
307508,1.0,0.535722,0.218859,-14966.0,0.286998,0.117248,-8017.611736,0.047899,-3275.445025,223981156.0,0.153751,0.062812,-4295.209004,0.025661,-1754.727146,1.199916e+08,0.010483,-716.860892,4.902031e+07,-3.352102e+12
307509,1.0,0.514163,0.661024,-11961.0,0.264363,0.339874,-6149.901495,0.436952,-7906.502552,143065521.0,0.135926,0.174750,-3162.050698,0.224665,-4065.229651,7.355897e+07,0.288836,-5226.384299,9.456968e+07,-1.711207e+12


In [26]:
#df = pd.concat([df,poly_features],axis=1)
df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251.0,0.0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,-8456.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252.0,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,365243.0,-4388.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
307508,456253.0,0.0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,-6737.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
307509,456254.0,0.0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,-2562.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
# Feature engineering manual features based on dataset columns and credit eligibility found on internet
df['MONTHLY_INCOME'] = df['AMT_INCOME_TOTAL']/12
df['MONTHLY_CREDIT'] = df['AMT_ANNUITY']/12
df['LENGTH_CREDIT'] = df['AMT_CREDIT']/df['AMT_ANNUITY']
df['PERCENTAGE_SALARY'] = df['MONTHLY_CREDIT']/df['MONTHLY_INCOME']
df['DELTA_LOAN_GOOD'] = df['AMT_CREDIT']-df['AMT_GOODS_PRICE']
df['HOUSE_INCOME'] = df.apply(
    lambda row: row['MONTHLY_INCOME'] / (row['CNT_CHILDREN'] + 1) if row['CNT_CHILDREN'] > 0 else row['MONTHLY_INCOME'],
    axis=1
)
df['AGE'] = df['DAYS_BIRTH'].abs()/365
df['YEARS_EMPLOYED'] = df['DAYS_EMPLOYED'].abs()/365
df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE,MONTHLY_INCOME,MONTHLY_CREDIT,LENGTH_CREDIT,PERCENTAGE_SALARY,DELTA_LOAN_GOOD,HOUSE_INCOME,AGE,YEARS_EMPLOYED
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,1.0,0.0,16875.0,2058.375,16.461104,0.121978,55597.5,16875.0,25.920548,1.745205
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0.0,0.0,22500.0,2974.875,36.234085,0.132217,164002.5,22500.0,45.931507,3.254795
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,1.0,0.0,5625.0,562.500,20.000000,0.100000,0.0,5625.0,52.180822,0.616438
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,1.0,0.0,11250.0,2473.875,10.532818,0.219900,15682.5,11250.0,52.068493,8.326027
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,1.0,0.0,10125.0,1822.125,23.461618,0.179963,0.0,10125.0,54.608219,8.323288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251.0,0.0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,-8456.0,...,0.0,0.0,13125.0,2296.500,9.242325,0.174971,29700.0,13125.0,25.553425,0.646575
307507,456252.0,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,365243.0,-4388.0,...,1.0,0.0,6000.0,1000.125,22.459693,0.166687,44550.0,6000.0,56.917808,1000.665753
307508,456253.0,0.0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,-6737.0,...,1.0,0.0,12750.0,2498.250,22.604623,0.195941,92664.0,12750.0,41.002740,21.701370
307509,456254.0,0.0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,-2562.0,...,1.0,0.0,14250.0,1683.750,18.317595,0.118158,50607.0,14250.0,32.769863,13.112329


In [28]:
df.head(20)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE,MONTHLY_INCOME,MONTHLY_CREDIT,LENGTH_CREDIT,PERCENTAGE_SALARY,DELTA_LOAN_GOOD,HOUSE_INCOME,AGE,YEARS_EMPLOYED
0,100002.0,0.0,202500.000,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,1.0,0.0,16875.00000,2058.375,16.461104,0.121978,55597.5,16875.00000,25.920548,1.745205
1,100003.0,0.0,270000.000,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0.0,0.0,22500.00000,2974.875,36.234085,0.132217,164002.5,22500.00000,45.931507,3.254795
2,100004.0,0.0,67500.000,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,1.0,0.0,5625.00000,562.500,20.000000,0.100000,0.0,5625.00000,52.180822,0.616438
3,100006.0,0.0,135000.000,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,1.0,0.0,11250.00000,2473.875,10.532818,0.219900,15682.5,11250.00000,52.068493,8.326027
4,100007.0,0.0,121500.000,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,1.0,0.0,10125.00000,1822.125,23.461618,0.179963,0.0,10125.00000,54.608219,8.323288
5,100008.0,0.0,99000.000,490495.5,27517.5,454500.0,0.035792,-16941.0,-1588.0,-4970.0,...,1.0,0.0,8250.00000,2293.125,17.824857,0.277955,35995.5,8250.00000,46.413699,4.350685
6,100009.0,1.0,171000.000,1560726.0,41301.0,1395000.0,0.035792,-13778.0,-3130.0,-1213.0,...,1.0,0.0,14250.00000,3441.750,37.789061,0.241526,165726.0,7125.00000,37.747945,8.575342
7,100010.0,0.0,360000.000,1530000.0,42075.0,1530000.0,0.003122,-18850.0,-449.0,-4597.0,...,1.0,0.0,30000.00000,3506.250,36.363636,0.116875,0.0,30000.00000,51.643836,1.230137
8,100011.0,0.0,112500.000,1019610.0,33826.5,913500.0,0.018634,-20099.0,365243.0,-7427.0,...,1.0,0.0,9375.00000,2818.875,30.142344,0.300680,106110.0,9375.00000,55.065753,1000.665753
9,100012.0,0.0,135000.000,405000.0,20250.0,405000.0,0.019689,-14469.0,-2019.0,-14437.0,...,1.0,0.0,11250.00000,1687.500,20.000000,0.150000,0.0,11250.00000,39.641096,5.531507


In [29]:
# Combiner bureau + application_train

df = pd.merge(df,df_bureau_balance,on='SK_ID_CURR',how='left')
df


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CLIENT_ON_TIME_min,CLIENT_ON_TIME_sum,CLIENT_LATE_mean,CLIENT_LATE_max,CLIENT_LATE_min,CLIENT_LATE_sum,CLIENT_VERY_LATE_mean,CLIENT_VERY_LATE_max,CLIENT_VERY_LATE_min,CLIENT_VERY_LATE_sum
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0.0,18.0,0.045455,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251.0,0.0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,-8456.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252.0,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,365243.0,-4388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253.0,0.0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,-6737.0,...,0.0,12.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307509,456254.0,0.0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,-2562.0,...,0.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
print(df.shape)
print(y.shape)
print(df.isnull().sum())
#df = df[df['type_credit'].notna()]

df = pd.merge(df,df_previous,on='SK_ID_CURR',how=('left'),suffixes=('_x1', '_x2'))
#df = df[df['SK_DPD_DEF_y'].notna()]
df.shape

(307511, 436)
(307511, 2)
SK_ID_CURR                   0
CNT_CHILDREN                 0
AMT_INCOME_TOTAL             0
AMT_CREDIT                   0
AMT_ANNUITY                  0
                         ...  
CLIENT_LATE_sum          44020
CLIENT_VERY_LATE_mean    44020
CLIENT_VERY_LATE_max     44020
CLIENT_VERY_LATE_min     44020
CLIENT_VERY_LATE_sum     44020
Length: 436, dtype: int64


(307511, 1284)

In [31]:
df.isnull().sum()

SK_ID_CURR                                    0
CNT_CHILDREN                                  0
AMT_INCOME_TOTAL                              0
AMT_CREDIT                                    0
AMT_ANNUITY                                   0
                                          ...  
NAME_CONTRACT_STATUS_Sent proposal_min    16454
NAME_CONTRACT_STATUS_Signed_mean_y        16454
NAME_CONTRACT_STATUS_Signed_sum_y         16454
NAME_CONTRACT_STATUS_Signed_max_y         16454
NAME_CONTRACT_STATUS_Signed_min_y         16454
Length: 1284, dtype: int64

In [32]:
similar_features=[]
df = df.drop(columns=similar_features,axis=1)
df.shape

(307511, 1284)

In [39]:
def impute_in_chunks(df, chunk_size=100):
    # Remplacer les infinis par des NaNs
    df = df.replace([np.inf, -np.inf], np.nan)
    
    imputer = SimpleImputer(strategy='constant', fill_value=0)
    chunks = []

    for i in range(0, df.shape[1], chunk_size):
        chunk = df.iloc[:, i:i+chunk_size]
        chunk_imputed = pd.DataFrame(imputer.fit_transform(chunk), columns=chunk.columns)
        chunks.append(chunk_imputed)

    return pd.concat(chunks, axis=1)

In [40]:
df.isnull().sum()
#df = df[df['late_credit'].notna()]
df.isnull().sum()

#y = y[y['SK_ID_CURR'].isin(df['SK_ID_CURR'])]
print(y.shape)
print(df.shape)
y=y.drop('SK_ID_CURR',axis=1)

df  =  impute_in_chunks(df)
print(df.isnull().sum())
df.head()

(307511, 1)
(307511, 1284)
SK_ID_CURR                                0
CNT_CHILDREN                              0
AMT_INCOME_TOTAL                          0
AMT_CREDIT                                0
AMT_ANNUITY                               0
                                         ..
NAME_CONTRACT_STATUS_Sent proposal_min    0
NAME_CONTRACT_STATUS_Signed_mean_y        0
NAME_CONTRACT_STATUS_Signed_sum_y         0
NAME_CONTRACT_STATUS_Signed_max_y         0
NAME_CONTRACT_STATUS_Signed_min_y         0
Length: 1284, dtype: int64


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Refused_max_y,NAME_CONTRACT_STATUS_Refused_min_y,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Sent proposal_sum,NAME_CONTRACT_STATUS_Sent proposal_max,NAME_CONTRACT_STATUS_Sent proposal_min,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_Signed_sum_y,NAME_CONTRACT_STATUS_Signed_max_y,NAME_CONTRACT_STATUS_Signed_min_y
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
df

In [ ]:
df=df.drop('SK_ID_CURR',axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
import re

df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train,X_test,y_train,y_test = train_test_split(df,y,test_size=0.2)

#x_train = x_train.drop('SK_ID_CURR', axis=1)
#x_test = x_test.drop('SK_ID_CURR', axis=1)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np




model_sk = lgb.LGBMClassifier(boosting_type='gbdt', max_depth=7, learning_rate=0.01, n_estimators= 2000, 
                 class_weight='balanced', subsample=0.9, colsample_bytree= 0.8, n_jobs=-1)
train_features, valid_features, train_y, valid_y = train_test_split(X_train, y_train.values.ravel(), test_size = 0.15, random_state = 33)
model_sk.fit(train_features, train_y, callbacks=[
        lgb.early_stopping(stopping_rounds=200)],eval_set = [(valid_features, valid_y)], eval_metric = 'auc')

# Save the Selected column, Future use
feature_imp = pd.DataFrame(sorted(zip(model_sk.feature_importances_, X_train.columns)), columns=['Value','Feature'])
features_df = feature_imp.sort_values(by="Value", ascending=False)
selected_features = list(features_df[features_df['Value']>=50]['Feature'])
# Saving the selected features into pickle file


In [ ]:
import pickle

with open('select_features.txt','wb') as fp:
    pickle.dump(selected_features, fp)
print('The no. of features selected:',len(selected_features))



In [ ]:
from sklearn.metrics import roc_auc_score


weight = np.ones((len(X_train),), dtype=int)
for i in range(len(X_train)):
    if int(y_train.iloc[i])== 0:
        weight[i]=1
    else:
        weight[i]=11

train_data=lgb.Dataset(X_train[selected_features], label = y_train, weight= weight )
valid_data=lgb.Dataset(X_test[selected_features], label = y_test)
cv_auc_score = []
max_depth = [3, 5, 7, 10]
for i in max_depth:
    
    params = {'boosting_type': 'gbdt',
          'max_depth' : i,
          'objective': 'binary',
          'nthread': 5,
          'num_leaves': 32,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 0.7,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 20,
          'reg_lambda': 20,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 10,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'
          }
lgbm = lgb.train(params,
                 train_data,
                 2500,
                 valid_sets=valid_data,
                  callbacks=[lgb.early_stopping(stopping_rounds=100)],
                 )
y_pred_prob = lgbm.predict(X_test[selected_features])
cv_auc_score.append(roc_auc_score(y_test,y_pred_prob))
print('For  max_depth {0} and some other parameters, cross validation AUC score {1}'.format(i,roc_auc_score(y_test,y_pred_prob)))
print('The optimal  max_depth: ', max_depth[np.argmax(cv_auc_score)])
params = {'boosting_type': 'gbdt',
          'max_depth' : max_depth[np.argmax(cv_auc_score)],
          'objective': 'binary',
          'nthread': 5,
          'num_leaves': 32,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 0.7,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 20,
          'reg_lambda': 20,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 10,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'
          }
lgbm = lgb.train(params,
                 train_data,
                 2500,
                 valid_sets=valid_data,
                 callbacks=[lgb.early_stopping(stopping_rounds=100)],
                 )
y_pred_prob = lgbm.predict(X_train[selected_features])
print('For best max_depth {0}, The Train AUC score is {1}'.format(max_depth[np.argmax(cv_auc_score)], 
                                                                  roc_auc_score(y_train,y_pred_prob) ))    

y_pred_prob = lgbm.predict(X_test[selected_features])
print('For best max_depth {0}, The Test AUC score is {1}'.format(max_depth[np.argmax(cv_auc_score)], 
                                                                 roc_auc_score(y_test,y_pred_prob) ))
y_pred = np.ones((len(X_test),), dtype=int)
for i in range(len(y_pred_prob)):
    if y_pred_prob[i]<=0.5:
        y_pred[i]=0
    else:
        y_pred[i]=1

In [ ]:
#top_features.shape

In [ ]:

# Import application_test

df_test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
df_test.head()
print(df_test.shape)
df_test

In [ ]:

test_categorical_columns = df_test.select_dtypes(exclude='number').columns
test_df_categorical = df_test[test_categorical_columns].copy()
hot_encoded_features = pd.DataFrame(hot_encoder.transform(test_df_categorical[hot]),columns=hot_encoder.get_feature_names_out(hot))
ordinal_encoded_features = pd.DataFrame(encoder.transform(test_df_categorical[label]),columns=label)

df_categorical_test = pd.concat([hot_encoded_features,ordinal_encoded_features],axis=1)
df_categorical_test


In [ ]:
numerical_columns = df_test.select_dtypes(include='number').columns
test_df_numerical = df_test[numerical_columns].copy()

df_test = pd.concat([test_df_numerical,df_categorical_test],axis=1)
df_test

In [ ]:
df_test = df_test.drop(columns=empty_columns,axis=1)

In [ ]:
df_test = pd.DataFrame(imp_mean.transform(df_test),columns = df_test.columns)
df_test

In [ ]:
poly_features_test = df_test[[ 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
poly_features_test = poly_transformer.transform(poly_features_test)
poly_features_test = pd.DataFrame(poly_features_test, 
                             columns = poly_transformer.get_feature_names_out([ 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))
poly_features_test

In [ ]:
#df_test = df_test.drop(columns=['EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'],axis=1)
#df_test = pd.concat([df_test,poly_features_test],axis=1)
df_test

In [ ]:
# Feature engineering manual features based on dataset columns and credit eligibility found on internet
df_test['MONTHLY_INCOME'] = df_test['AMT_INCOME_TOTAL']/12
df_test['MONTHLY_CREDIT'] = df_test['AMT_ANNUITY']/12
df_test['LENGTH_CREDIT'] = df_test['AMT_CREDIT']/df_test['AMT_ANNUITY']
df_test['PERCENTAGE_SALARY'] = df_test['MONTHLY_CREDIT']/df_test['MONTHLY_INCOME']
df_test['DELTA_LOAN_GOOD'] = df_test['AMT_CREDIT']-df_test['AMT_GOODS_PRICE']
df_test['HOUSE_INCOME'] = df_test.apply(
    lambda row: row['MONTHLY_INCOME'] / (row['CNT_CHILDREN'] + 1) if row['CNT_CHILDREN'] > 0 else row['MONTHLY_INCOME'],
    axis=1
)
df_test['AGE'] = df_test['DAYS_BIRTH'].abs()/365
df_test['YEARS_EMPLOYED'] = df_test['DAYS_EMPLOYED'].abs()/365
df_test

In [ ]:
print("Before:")
print(df_test.memory_usage(deep=True).sum() / 1024**2, "MB")

df_test = force_optimize_all_to_int(df_test)

print("After:")
print(df_test.memory_usage(deep=True).sum() / 1024**2, "MB")


In [ ]:
df_test = pd.merge(df_test,df_bureau_balance,on='SK_ID_CURR',how='left')



In [ ]:
df_test = pd.merge(df_test,df_previous,on='SK_ID_CURR',how='left',suffixes=('_x1', '_x2'))
df_test = df_test.drop(columns=similar_features,axis=1)

In [ ]:
df_test
df_test.index=df_test['SK_ID_CURR']
df_test

In [ ]:

df_test.isnull().sum()
df_test = impute_in_chunks(df_test)

df_test.head()


df_test.index = df_test['SK_ID_CURR']
df_test = df_test.drop('SK_ID_CURR', axis=1)


In [ ]:
df_test

In [ ]:
df_test = df_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
df_test = df_test[selected_features]

In [ ]:
#temp2 = df_test['SK_ID_CURR']
y_test_predicted = lgbm.predict(df_test)

In [ ]:
from pathlib import Path  
# Chemin complet vers le fichier
filepath = Path('/kaggle/working/submission.csv')

columns = df_test.index.astype(str).str.replace('.0', '').astype('Int32')
df_output = pd.DataFrame({
    'SK_ID_CURR': columns,  # REQUIRED column
    'TARGET': y_test_predicted             # Your predictions
})

# Sauvegarde en CSV
df_output.to_csv(filepath, index=False)

import os
print(os.listdir('/kaggle/working/'))  

In [ ]:
df_output